In [ ]:
import numpy as np
import pandas as pd

# Uses `Distribution`, `run_de`, and `fitness` in the file
from method import *

# Clean up the output
np.set_printoptions(precision=2, suppress=True)

# Random seed for reproducibility
SEED = 20241225

## Real data experiments in the paper
This file illustrates how to replicate the real data experiments, which is very similar to the instruction given in `Readme.md`

Assume that the CGM data from Shah et al. (2019) and Brown et al. (2019) are accessed and processed as guided in the [Awesome-CGM](https://github.com/IrinaStatsLab/Awesome-CGM) repository, saved in `./data/shah2019_filtered.csv` and `./data/brown2019_filtered.csv`. 

These datasets are measured by Dexcom G6, having glucose ranges from 39 mg/dL to 401 mg/dL.

As the differential evolution (DE) is a stochastic algorithm, we fix `SEED=20241225` to reproduce the experimental results illustrated in the paper. 

In [ ]:
# Load data
data_shah = pd.read_csv("./data/shah2019_filtered.csv")
grouped_data_shah = data_shah.groupby('id').agg({'gl': list}).reset_index()
data_class_shah = Distribution(grouped_data_shah["gl"], ran=(39., 401.), M=200)

data_brown = pd.read_csv("./data/brown2019_filtered.csv")
grouped_data_brown = data_brown.groupby('id').agg({'gl': list}).reset_index()
data_class_brown = Distribution(grouped_data_brown["gl"], ran=(39., 401.), M=200)

After making data into `Distribution` classes, we can run `run_de` with specified target number of thresholds `K` and the threshold-optimality criteria: `loss="Loss1"` or `loss="Loss2"`. 

If you want to see the optimization progress, set `disp=True` in the `run_de` function. The function typically takes a couple of minutes, depending on the device.

Experiments conducted in the paper are given below.

### Shah dataset

##### $L_1$ loss

In [ ]:
# Selecting K=4 thresholds
best_cutoffs, min_loss = run_de(data_class_shah, K=4, loss="Loss1", seed=SEED)
print("Cutoffs:", best_cutoffs[1:-1])
print("Obtained loss:", min_loss)

Cutoffs: [ 75.83 100.69 123.7  154.96]
Obtained loss: 16.619439163631053


In [ ]:
# Selecting K=2 thresholds
best_cutoffs, min_loss = run_de(data_class_shah, K=2, loss="Loss1", seed=SEED)
print("Cutoffs:", best_cutoffs[1:-1])
print("Obtained loss:", min_loss)

Cutoffs: [ 71.69 127.91]
Obtained loss: 88.90773604650336


##### $L_2$ loss (supplementary)

In [ ]:
# Selecting K=4 thresholds
best_cutoffs2, min_loss2 = run_de(data_class_shah, K=4, loss="Loss2", seed=SEED)
print("Cutoffs:", best_cutoffs2[1:-1])
print("Obtained loss (x 10^{-3}):", min_loss2 / 1000)

Cutoffs: [ 76.73 106.9  138.1  176.66]
Obtained loss (x 10^{-3}): 0.7120189776106792


In [ ]:
# Selecting K=2 thresholds
best_cutoffs2, min_loss2 = run_de(data_class_shah, K=2, loss="Loss2", seed=SEED)
print("Cutoffs:", best_cutoffs2[1:-1])
print("Obtained loss (x 10^{-3}):", min_loss2 / 1000)

Cutoffs: [123.58 163.03]
Obtained loss (x 10^{-3}): 11.585229903028312


### Brown dataset

##### $L_1$ loss

In [ ]:
# Selecting K=4 thresholds
best_cutoffs, min_loss = run_de(data_class_brown, K=4, loss="Loss1", seed=SEED)
print("Cutoffs:", best_cutoffs[1:-1])
print("Obtained loss:", min_loss)

Cutoffs: [ 84.88 171.2  232.62 301.58]
Obtained loss: 41.215727144747326


In [ ]:
# Selecting K=2 thresholds
best_cutoffs, min_loss = run_de(data_class_brown, K=2, loss="Loss1", seed=SEED)
print("Cutoffs:", best_cutoffs[1:-1])
print("Obtained loss:", min_loss)

Cutoffs: [210.48 288.43]
Obtained loss: 398.2131637666346


##### Semi-supervised, fixing 70 and 181 mg/dL

In [ ]:
# Selecting K=2 additional thresholds (4 thresholds total)
best_cutoffs, min_loss = run_de(data_class_brown, K=2, loss="Loss1", seed=SEED, fixed=(70, 181))
print("Cutoffs:", best_cutoffs[1:-1])
print("Obtained loss:", min_loss)

Cutoffs: [ 70.   181.   240.9  305.95]
Obtained loss: 64.05054061137514


##### $L_2$ loss (supplementary)

In [ ]:
# Selecting K=4 thresholds
best_cutoffs2, min_loss2 = run_de(data_class_brown, K=4, loss="Loss2", seed=SEED)
print("Cutoffs:", best_cutoffs2[1:-1])
print("Obtained loss (x 10^{-3}):", min_loss2 / 1000)

Cutoffs: [168.89 249.13 294.49 351.83]
Obtained loss (x 10^{-3}): 30.808015190380164


In [ ]:
# Selecting K=2 thresholds
best_cutoffs2, min_loss2 = run_de(data_class_brown, K=2, loss="Loss2", seed=SEED)
print("Cutoffs:", best_cutoffs2[1:-1])
print("Obtained loss (x 10^{-3}):", min_loss2 / 1000)

Cutoffs: [184.69 284.77]
Obtained loss (x 10^{-3}): 63.28056823880764


### Combined data

In [ ]:
data_concat = pd.concat([grouped_data_shah, grouped_data_brown], ignore_index=True)
data_class_concat = Distribution(data_concat["gl"], ran=(39., 401.), M=200)

##### $L_2$ loss

Selecting $K=2$ thresholds

In [ ]:
# Selecting K=2 thresholds
best_cutoffs2, min_loss2 = run_de(data_class_concat, K=2, loss="Loss2", seed=SEED)
print("Cutoffs:", best_cutoffs2[1:-1])
print("Obtained loss (x 10^{-3}):", min_loss2 / 1000)

Cutoffs: [148.02 255.87]
Obtained loss (x 10^{-3}): 395.39153325586994


$K=4$ (supplementary)

In [ ]:
# Selecting K=4 thresholds
best_cutoffs2, min_loss2 = run_de(data_class_concat, K=4, loss="Loss2", seed=SEED)
print("Cutoffs:", best_cutoffs2[1:-1])
print("Obtained loss (x 10^{-3}):", min_loss2 / 1000)

Cutoffs: [127.75 192.86 246.25 311.57]
Obtained loss (x 10^{-3}): 154.46696259920486


##### $L_1$ loss (supplementary)

In [ ]:
# Selecting K=2 thresholds
best_cutoffs, min_loss = run_de(data_class_concat, K=2, loss="Loss1", seed=SEED)
print("Cutoffs:", best_cutoffs[1:-1])
print("Obtained loss:", min_loss)

Cutoffs: [132.34 246.28]
Obtained loss: 309.3412821689561


In [ ]:
# Selecting K=4 thresholds
best_cutoffs, min_loss = run_de(data_class_concat, K=4, loss="Loss1", seed=SEED)
print("Cutoffs:", best_cutoffs[1:-1])
print("Obtained loss:", min_loss)

Cutoffs: [ 76.16 123.44 194.02 279.28]
Obtained loss: 66.98148242930007


### Optimality measures at the traditional thresholds

In [ ]:
print("Shah dataset")
print("    L1 at two traditional:", fitness([70, 181], data_class_shah, loss="Loss1"))
print("    L1 at four traditional:", fitness([54, 70, 181, 251], data_class_shah, loss="Loss1"))

# Precompute the Wasserstein distance matrix for L2 loss calculation
data_class_shah.Wdist_matrix()
print("    L2 at two traditional:", fitness([70, 181], data_class_shah, loss="Loss2") / 1000, "(multiplied by 10^{-3})")
print("    L2 at four traditional:", fitness([54, 70, 181, 251], data_class_shah, loss="Loss2") / 1000, "(multiplied by 10^{-3})")

print("\nBrown dataset")
print("    L1 at two traditional:", fitness([70, 181], data_class_brown, loss="Loss1"))
print("    L1 at four traditional:", fitness([54, 70, 181, 251], data_class_brown, loss="Loss1"))
data_class_brown.Wdist_matrix()
print("    L2 at two traditional:", fitness([70, 181], data_class_brown, loss="Loss2") / 1000, "(multiplied by 10^{-3})")
print("    L2 at four traditional:", fitness([54, 70, 181, 251], data_class_brown, loss="Loss2") / 1000, "(multiplied by 10^{-3})")

print("\nConcatenated dataset")
data_class_concat.Wdist_matrix()
print("    L2 at two traditional:", fitness([70, 181], data_class_concat, loss="Loss2") / 1000, "(multiplied by 10^{-3})")
print("    L2 at four traditional:", fitness([54, 70, 181, 251], data_class_concat, loss="Loss2") / 1000, "(multiplied by 10^{-3})")
print("    L1 at two traditional:", fitness([70, 181], data_class_concat, loss="Loss1"))
print("    L1 at four traditional:", fitness([54, 70, 181, 251], data_class_concat, loss="Loss1"))

Shah dataset
    L1 at two traditional: 656.8785795451477
    L1 at four traditional: 655.939314472095
    L2 at two traditional: 28.32808440253849 (multiplied by 10^{-3})
    L2 at four traditional: 28.801864648312193 (multiplied by 10^{-3})

Brown dataset
    L1 at two traditional: 1235.9789988634705
    L1 at four traditional: 159.90153344550367
    L2 at two traditional: 1450.9149694569626 (multiplied by 10^{-3})
    L2 at four traditional: 92.74154648603994 (multiplied by 10^{-3})

Concatenated dataset
    L2 at two traditional: 2430.0156676593133 (multiplied by 10^{-3})
    L2 at four traditional: 2971.452506196784 (multiplied by 10^{-3})
    L1 at two traditional: 946.428789204309
    L1 at four traditional: 407.92042395879935
